# Track A Computer Vision - Image Classification

## M0 : Quick Start

### Confirm GPU is ready

In [1]:
!nvidia-smi || echo "nvidia-smi unavailable (CPU runtime)"

Sat Nov 15 09:14:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 577.03                 Driver Version: 577.03         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8              2W /   50W |     231MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Point the notebook at the project folder

In [2]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent.resolve()
elif PROJECT_ROOT.name == "content":
    candidate = PROJECT_ROOT / "od-project"
    if candidate.exists():
        PROJECT_ROOT = candidate.resolve()

if not (PROJECT_ROOT / "src").exists():
    raise FileNotFoundError(
        f"Could not locate project root at {PROJECT_ROOT}. Upload or clone od-project before proceeding."
    )

os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT / "src") not in sys.path:
    sys.path.append(str(PROJECT_ROOT / "src"))
print(f"Project root: {PROJECT_ROOT}")

Project root: C:\Users\lucas\End-to-End-Deep-Learning-Systems\End-to-End-Deep-Learning-Systems\starters\cv-project-starter\cv-project


### Install the project requirement

In [3]:
!pip install -r requirements.txt

### Téléchargement du dataset

In [4]:
from torchvision.datasets import OxfordIIITPet

ds = OxfordIIITPet(
    root="./data/pets",
    split="trainval",
    target_types="category",
    download=True
)

### Run the smoke test

In [5]:
from src import smoke_check

smoke_path = smoke_check.run_smoke("configs/cv_oxfordpet.yaml")
print(smoke_path.read_text())

{
  "loss": 4.0178351402282715,
  "batch_size": 32,
  "num_classes": 37,
  "device": "cuda"
}


# M1: Problem Scoping & Data Validation

## 1.1. Problem Definition

**Task (Track A)**  
Nous voulons entraîner un modèle de **classification d’images** pour reconnaître la race d’un animal (chat ou chien) à partir d’une photo.

- **Input (X)** : images couleur RGB de chats et de chiens, format tensoriel `3 × H × W` (après préprocessing, redimensionnées à `224 × 224` et normalisées avec les statistiques ImageNet).  
- **Output (y)** : un entier dans `0, …, 36` représentant l’une des **37 classes** (races de chats & chiens).  
- **Type de problème** : classification supervisée multi-classe.

## 1.2. Evaluation Metrics

Nous suivons les métriques demandées dans le sujet :

- **Accuracy** : proportion de prédictions correctes sur l’ensemble de validation/test.
- **Macro-F1 Score** : moyenne du F1-score calculé indépendamment pour chaque classe (utile en cas de classes déséquilibrées).
- **Matrice de confusion** : permet de voir quelles races sont souvent confondues entre elles.

L’accuracy donne une vue globale, tandis que le macro-F1 met l’accent sur les classes minoritaires. La matrice de confusion servira plus tard pour l’analyse d’erreurs (M4).


## 1.3. Data Card – Oxford-IIIT Pet

**Nom du dataset**  
Oxford-IIIT Pet Dataset

**Source**  
Visual Geometry Group (VGG), University of Oxford.  
Site officiel : *Oxford-IIIT Pet Dataset* (O. M. Parkhi et al., 2012).

**Description générale**  
- ~7 000 images de **chats et de chiens** en conditions naturelles (intérieur, extérieur, lumière variable…).
- **37 classes** correspondant à différentes races (25 races de chiens + 12 races de chats).
- Images de résolution et de ratio variables, centrées approximativement sur l’animal, avec arrière-plans variés.

**Taille & splits**  
Dans ce projet, nous utilisons les splits officiels et un split validation interne :

- Split officiel `trainval` fourni par le dataset.
- Split officiel `test`.
- À partir de `trainval`, nous créons un split **train/val** :  
  - `train` ≈ 90% de trainval  
  - `val` ≈ 10% de trainval  
Le `test` reste celui fourni par les auteurs, non utilisé pendant l’entraînement.

**Caractéristiques des données**  
- **Entrées** : images couleur au format JPEG, converties en tensors PyTorch, redimensionnées à `224×224`, normalisées avec les moyennes / écarts-types d’ImageNet.
- **Labels** : entiers 0–36, mappés à des noms de classes (noms de races).  
- **Préprocessing / Data augmentation** :
  - Redimensionnement / recadrage aléatoire (`RandomResizedCrop`),
  - Flip horizontal aléatoire en entraînement,
  - Normalisation standard ImageNet.

**Licence & usage**  
- Dataset publié pour la recherche académique en vision par ordinateur.  
- Utilisation dans ce projet : **démo pédagogique / proof-of-concept**, sans déploiement en production ni usage commercial.

**Potentiels biais & limitations**  
- **Déséquilibre de classes** : certaines races sont probablement sur-représentées par rapport à d’autres.
- **Biais de contexte** : photos principalement issues de contextes domestiques occidentaux, peu de diversité géographique, culturelle ou de conditions extrêmes.
- **Variabilité de la qualité** : résolution, éclairage, flou, occlusions… peuvent influencer la performance du modèle.
- **Généralisation limitée** : un modèle entraîné sur ce dataset pourrait mal se comporter sur des photos prises dans d’autres conditions (qualité smartphone très basse, angles extrêmes, animaux partiellement visibles, etc.).

**Considérations éthiques**  
- Modèle sans impact direct sur des humains, mais les biais de représentativité peuvent fausser l’interprétation de la performance s’il était utilisé pour des applications réelles.
- Toute utilisation en production (ex. app de reconnaissance de races) devrait être accompagnée de disclaimers sur les limites et le contexte d’entraînement du modèle.


## 1.4. Train / Validation / Test splits

Pour ce projet, nous souhaitons un schéma de splits **robuste et reproductible** :

1. **Train / Val** :  
   - On charge le split officiel `trainval` fourni par Oxford-IIIT Pet.  
   - On applique un split aléatoire (seed fixé) 90% / 10% pour obtenir `train` et `val`.  
   - Train est utilisé pour apprendre les paramètres du modèle.  
   - Val est utilisé pour le choix d’hyperparamètres, l’early stopping et le suivi des performances pendant l’entraînement.

2. **Test** :  
   - On utilise le split officiel `test` fourni par le dataset.  
   - Il n’est jamais utilisé pendant l’entraînement ou le tuning.  
   - Il sert uniquement à **mesurer la performance finale** du modèle (généralisation).

Les splits sont implémentés dans `src/data.py` via la fonction `build_dataloaders`, qui :
- charge `OxfordIIITPet(split="trainval")`,
- effectue un `random_split` selon `val_split` dans le fichier de configuration,
- applique des transforms différentes pour train (avec augmentation) et val/test (déterministes).

In [6]:
from yaml import safe_load
from src.data import build_dataloaders
from torchvision.datasets import OxfordIIITPet
from collections import Counter
from pathlib import Path

cfg = safe_load(open("configs/cv_oxfordpet.yaml", encoding="utf-8"))

# Dataloaders train/val (via notre pipeline projet)
train_loader, val_loader, num_classes, classes = build_dataloaders(cfg)

print("=== Splits internes (train/val) ===")
print("Num classes :", num_classes)
print("Taille train :", len(train_loader.dataset))
print("Taille val   :", len(val_loader.dataset))
print("Exemples de classes :", classes[:10])

# Charger le split test officiel juste pour vérifier sa taille
root = Path(cfg["data"]["root"])
test_set = OxfordIIITPet(
    root=str(root),
    split="test",
    target_types="category",
    download=False,
    transform=None,
)
print("Taille test (officiel) :", len(test_set))

# Vérifier un peu la distribution des labels sur le train
all_labels = []
for _, targets in train_loader:
    all_labels.extend(targets.tolist())

label_counts = Counter(all_labels)
print("\nNombre d'images par classe (train) pour les 5 premières classes :")
for cls_idx in list(label_counts.keys())[:5]:
    print(f"  classe {cls_idx:2d} ({classes[cls_idx]:>15}) : {label_counts[cls_idx]} images")


=== Splits internes (train/val) ===
Num classes : 37
Taille train : 3312
Taille val   : 368
Exemples de classes : ['Abyssinian', 'American Bulldog', 'American Pit Bull Terrier', 'Basset Hound', 'Beagle', 'Bengal', 'Birman', 'Bombay', 'Boxer', 'British Shorthair']
Taille test (officiel) : 3669

Nombre d'images par classe (train) pour les 5 premières classes :
  classe 29 (        Samoyed) : 84 images
  classe  0 (     Abyssinian) : 93 images
  classe 16 (       Havanese) : 93 images
  classe  8 (          Boxer) : 90 images
  classe  5 (         Bengal) : 92 images


# M2 – Baseline Model Implementation

L'objectif de cette section est d'établir un **baseline** pour le modèle choisi, c’est-à-dire une première version fonctionnelle du réseau sans optimisation avancée.  

Les buts sont :
- Vérifier que le modèle peut traiter un batch complet sans erreur.
- Confirmer que les dataloaders fonctionnent.
- Effectuer un premier entraînement court (5 epochs) pour obtenir des métriques initiales.
- Établir un point de comparaison pour les améliorations du M3 et les ablations du M4.


## 2.1. Forward Pass Test (One Full Batch)

Nous construisons le modèle (ResNet-18 pré-entraîné) et vérifions qu’un batch complet issu du dataloader Oxford-IIIT Pet passe dans le réseau sans erreur.  
C’est une étape essentielle pour s'assurer que :
- le dataset est bien chargé,
- les transforms fonctionnent,
- les dimensions des images correspondent aux attentes,
- la tête de classification est correctement redimensionnée à 37 classes.

In [7]:
from yaml import safe_load
import torch
import torch.nn as nn

from src.data import build_dataloaders
from src.model import build_model

# Charger la config Oxford Pet
cfg = safe_load(open("configs/cv_oxfordpet.yaml", encoding="utf-8"))

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device =", device)

# Dataloaders + infos
train_loader, val_loader, num_classes, classes = build_dataloaders(cfg)
print("Num classes =", num_classes)
print("Taille train =", len(train_loader.dataset))
print("Taille val   =", len(val_loader.dataset))

# Construire le modèle baseline (ResNet-18) à partir de la config
model = build_model(cfg, num_classes).to(device)
criterion = nn.CrossEntropyLoss()

# Récupérer un batch
images, targets = next(iter(train_loader))
images = images.to(device)
targets = targets.to(device)

# Forward pass sur un batch complet
with torch.no_grad():
    outputs = model(images)
    loss = criterion(outputs, targets)

print("Images shape :", images.shape)      # [batch_size, 3, 224, 224]
print("Logits shape :", outputs.shape)     # [batch_size, 37]
print("Batch loss   :", loss.item())


Device = cuda
Num classes = 37
Taille train = 3312
Taille val   = 368
Images shape : torch.Size([32, 3, 224, 224])
Logits shape : torch.Size([32, 37])
Batch loss   : 3.9777839183807373


## 2.2 Baseline Training (configuration rapide)

Avant de passer à l’optimisation (M3), nous réalisons un premier entraînement rapide
de 5 époques afin de valider que :

- le modèle apprend correctement,
- la loss décroît bien,
- les dataloaders fonctionnent,
- le pipeline complet (train → save best → metrics) est opérationnel.

Pour cela, nous utilisons une configuration dédiée :  
**`cv_oxfordpet_fast.yaml`**, qui reprend la même architecture mais avec seulement **5 epochs**.

Ce run constitue notre *baseline* :  
il donne une première estimation des performances sans optimisation ni tuning.

In [8]:
!python src/train.py --config configs/cv_oxfordpet_fast.yaml


Done. Best val acc: 0.9131. Checkpoint: outputs\best.pt



train: 100%|██████████| 104/104 [00:25<00:00, 14.28it/s]
                                                        

val:  75%|███████▌  | 9/12 [00:17<00:03,  1.19s/it]
                                                   

train:  99%|█████████▉| 103/104 [00:26<00:00, 13.88it/s]
                                                        

val:  83%|████████▎ | 10/12 [00:21<00:02,  1.31s/it]
                                                    

train:  99%|█████████▉| 103/104 [00:26<00:00, 13.98it/s]
                                                        

val:  75%|███████▌  | 9/12 [00:17<00:03,  1.18s/it]
                                                   

train:  99%|█████████▉| 103/104 [00:24<00:00, 14.03it/s]
                                                        

val:  75%|███████▌  | 9/12 [00:17<00:03,  1.18s/it]
                                                   

train:  99%|█████████▉| 103/104 [00:25<00:00, 14.00it/s]
                                                        

va

## 2.3. Baseline Metrics

Après l'entraînement rapide, nous lisons les métriques obtenues (accuracy et macro-F1) afin d’établir une ligne de départ claire pour l’analyse future.


In [10]:
import json

with open("outputs/metrics.json", "r") as f:
    metrics = json.load(f)

print("=== Baseline metrics ===")
for k, v in metrics.items():
    print(f"{k}: {v}")


=== Baseline metrics ===
best_val_acc: 0.9131206274032593
classes: ['Abyssinian', 'American Bulldog', 'American Pit Bull Terrier', 'Basset Hound', 'Beagle', 'Bengal', 'Birman', 'Bombay', 'Boxer', 'British Shorthair', 'Chihuahua', 'Egyptian Mau', 'English Cocker Spaniel', 'English Setter', 'German Shorthaired', 'Great Pyrenees', 'Havanese', 'Japanese Chin', 'Keeshond', 'Leonberger', 'Maine Coon', 'Miniature Pinscher', 'Newfoundland', 'Persian', 'Pomeranian', 'Pug', 'Ragdoll', 'Russian Blue', 'Saint Bernard', 'Samoyed', 'Scottish Terrier', 'Shiba Inu', 'Siamese', 'Sphynx', 'Staffordshire Bull Terrier', 'Wheaten Terrier', 'Yorkshire Terrier']


## 2.4. Conclusion du Baseline

Le premier entraînement rapide sur 5 époques confirme que l’ensemble du pipeline fonctionne parfaitement.  
Le modèle a non seulement effectué le forward pass sur un batch complet sans aucune erreur, mais l’entraînement s’est déroulé de manière fluide, avec une décroissance normale de la loss et des métriques cohérentes.

Nous obtenons une **accuracy de validation de 91.31%**, ce qui constitue un **excellent résultat pour un simple baseline**, surtout en si peu d’époques.  
Cette performance montre que :

- les dataloaders et le préprocessing sont correctement configurés,  
- la tête de classification à 37 classes est bien intégrée,  
- le modèle pré-entraîné (**ResNet-18**) s'adapte rapidement au dataset Oxford-IIIT Pet,  
- la sauvegarde du meilleur modèle (`best.pt`) fonctionne comme prévu.




# M3 — Optimisation & Régularisation

Dans cette section, nous cherchons à améliorer notre baseline en intégrant plusieurs techniques d’optimisation et de régularisation.  
L’objectif est d’obtenir de meilleures performances que le baseline (val_acc ≈ 91.31%) obtenu en seulement 5 époques.

## 3.1. Stratégies d’optimisation et de régularisation utilisées

### **Weight Decay (L2 Regularization)**
Permet de pénaliser les poids trop grands afin de limiter l’overfitting.  
Déjà activé dans le YAML via :  
`weight_decay: 1e-4`

### **Scheduler (StepLR)**
Le taux d’apprentissage est réduit d’un facteur `gamma = 0.1` toutes les `step_size = 10` époques.  
Cela permet au modèle de faire de grands progrès au début, puis de se stabiliser.

### **Early Stopping**
Arrête l’entraînement si la performance de validation ne progresse plus après plusieurs époques.  
Évite le surapprentissage et accélère l’entraînement.

### **Fine-tuning complet du ResNet-18**
Contrairement à un simple entraînement de la dernière couche (freezing),  
nous entraînons **tout le réseau**, ce qui permet d’adapter les représentations internes au dataset Oxford-IIIT Pet.

## 3.2. Entraînement complet avec la configuration optimisée

Nous utilisons maintenant la configuration *principale* :  
**`cv_oxfordpet.yaml`**  
qui entraîne le modèle pendant 20 époques (avec scheduler + early stopping).

L’objectif est de :
- dépasser les performances du baseline (91.31%),
- obtenir un modèle plus robuste,
- générer des courbes d’apprentissage exploitables pour M4 et M5.

In [11]:
!python src/train.py --config configs/cv_oxfordpet.yaml

Done. Best val acc: 0.9131. Checkpoint: outputs\best.pt



train:  99%|█████████▉| 103/104 [00:26<00:00, 13.99it/s]
                                                        

val:  83%|████████▎ | 10/12 [00:17<00:02,  1.05s/it]
                                                    

train:  99%|█████████▉| 103/104 [00:23<00:00, 14.06it/s]
                                                        

val:  92%|█████████▏| 11/12 [00:16<00:00,  1.45it/s]
                                                    

train:  99%|█████████▉| 103/104 [00:24<00:00, 14.02it/s]
                                                        

val:  83%|████████▎ | 10/12 [00:16<00:02,  1.00s/it]
                                                    

train:  99%|█████████▉| 103/104 [00:29<00:00, 13.97it/s]
                                                        

val:  83%|████████▎ | 10/12 [00:20<00:02,  1.20s/it]
                                                    

train:  99%|█████████▉| 103/104 [00:26<00:00, 14.02it/s]
                                                      

## 3.3. Analyse des courbes loss / accuracy

Après l’entraînement, nous visualisons :
- la perte d’entraînement,
- la perte de validation,
- l’accuracy de validation,
- l’impact du scheduler sur la convergence.

Cela nous permettra d’évaluer l’efficacité des techniques d’optimisation et de confirmer que le modèle se stabilise correctement.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

log = pd.read_csv("outputs/log.csv")

plt.figure(figsize=(16,5))

plt.subplot(1,3,1)
plt.plot(log["epoch"], log["train_loss"], label="Train Loss")
plt.plot(log["epoch"], log["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss curves")
plt.legend()

plt.subplot(1,3,2)
plt.plot(log["epoch"], log["val_acc"], marker="o")
plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.title("Validation Accuracy")

plt.subplot(1,3,3)
plt.plot(log["epoch"], log["lr"], label="Learning Rate")
plt.xlabel("Epoch")
plt.ylabel("LR")
plt.title("Learning Rate Scheduler")
plt.legend()

plt.tight_layout()
plt.show()
